In [1]:
import numpy as np
import glob

import antoine
import water
import mmw

tmin = 1.0
dt   = 1.0


In [2]:
def write_csv(gas,ts,ps):
    X = np.array([ts,ps]).T 
    head = "T [K] , P_sat [Pa]"
    fpath = "sat/dat/%s_sat.csv"%gas 

    np.savetxt(fpath, X, fmt="%.9e", delimiter=',', header=head)

def write_tripcrit(gas, t_trip, t_crit):

    X = [t_trip]
    head = "T_trip [K]"
    fpath = "sat/dat/%s_trip.csv"%gas 
    np.savetxt(fpath, X, fmt="%.9e", delimiter=',', header=head)

    X = [t_crit]
    head = "T_crit [K]"
    fpath = "sat/dat/%s_crit.csv"%gas 
    np.savetxt(fpath, X, fmt="%.9e", delimiter=',', header=head)

In [3]:
# Main gases
for gas in antoine.gases.keys():
    gdict = antoine.gases[gas]

    t_arr = np.arange(tmin, gdict["T_crit"], dt)
    t_arr = np.concatenate((t_arr, [gdict["T_crit"]]))
    p_arr = [antoine.evaluate(t, gdict) for t in t_arr]
    write_csv(gas, t_arr, p_arr)
    write_tripcrit(gas, gdict["T_trip"], gdict["T_crit"])


In [4]:
# Water
t_arr = np.arange(tmin, water.T_crit, dt)
t_arr = np.concatenate((t_arr, [water.T_crit]))
arr_p = [water.psat_both(t) for t in t_arr]

write_csv("H2O",t_arr, arr_p)
write_tripcrit("H2O", water.T_trip, water.T_crit)